In [134]:
import torch
import re
import matplotlib.pyplot as plt

names = open('/Users/felipeakiomatsuoka/Desktop/NLP/DrugNames.txt', 'r').read().splitlines()

In [135]:
names = [re.sub(r'[^a-zA-Z0-9 ]', '', name) for name in names]  # remove special characters
names = [name.lower() for name in names]  # lower case

In [100]:
names[1:4]


['SULFAPYRIDINE', 'LIQUAEMIN SODIUM', 'LIQUAEMIN SODIUM']

In [136]:
b = {}
for n in names:
    chs = n + '.'
    for ch1, ch2 in zip(chs, chs[1:]):
        bigram = (ch1, ch2)
        b[bigram] = b.get(bigram, 0) + 1

In [111]:
chars = sorted(set(''.join(names)))
stoi = {s:i+1 for i, s in enumerate(chars)}
stoi['<E>'] = 0
itos = {i:s for s, i in stoi.items()}

In [112]:
N = torch.zeros((len(chars) + 1, len(chars) + 1), dtype=torch.int32)
for n in names:
    chs = ['<E>'] + list(n) + ['<E>']
    for ch1, ch2 in zip(chs, chs[1:]):
        idx1 = stoi[ch1]
        idx2 = stoi[ch2]
        N[idx1, idx2] += 1

In [143]:
g = torch.Generator().manual_seed(42)
P = (N+1).float()
P /= P.sum(dim=1, keepdim=True)

In [148]:
for i in range(10):
    out = []
    #ix = torch.randint(0, len(chars), (1,)).item()
    ix = 0   
    while True:
        p = P[ix]

        #p = torch.ones(len(chars) + 1) / len(chars) + 1

        ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
        if ix == 0:
            break
        out.append(itos[ix])

    print(''.join(out))

pr
ororinexloridee
roximpr
syde
hloride
drochydide pochlchydecumeroc samorosyciarinocipitisolochydatferolalobinacolocte
rospid depicarotasiume
zam hidete
oxenentibuon
tum amathipliuazone hlore


In [147]:
log_likelihood = 0
num = 0
for n in names:
    chs = ['<E>'] + list(n) + ['<E>']
    for ch1, ch2 in zip(n, n[1:]):
        idx1 = stoi[ch1]
        idx2 = stoi[ch2]
        prob = P[idx1, idx2]
        logprob = torch.log(prob)
        log_likelihood += logprob
        num += 1

print(f"Log likelihood: {log_likelihood:.2f}")
print(f"Negative and normalized log likelihood: {-log_likelihood / num:.2f}")

Log likelihood: -1715782.75
Negative and normalized log likelihood: 2.17
